# Analyse modelled fluxes

- Should also compare results of gridded NEE predictions (correlations etc.) with EC point data to see if they follow the same trends. 

In [ ]:
import sys
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from datacube.utils.dask import start_local_dask

In [ ]:
client = start_local_dask(mem_safety_margin='2Gb')
client

## Open predictor data

In [ ]:
data0 = xr.open_dataset('/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_2015.nc',
                       chunks=dict(x=1000,y=1000))
data1 = xr.open_dataset('/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_2016.nc',
                       chunks=dict(x=1000,y=1000))
data2 = xr.open_dataset('/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_2017.nc',
                       chunks=dict(x=1000,y=1000))
data3 = xr.open_dataset('/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_2018.nc',
                       chunks=dict(x=1000,y=1000))
data4 = xr.open_dataset('/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_2019.nc',
                       chunks=dict(x=1000,y=1000))
data5 = xr.open_dataset('/g/data/os22/chad_tmp/NEE_modelling/results/input_data/input_data_2020.nc',
                       chunks=dict(x=1000,y=1000))

In [ ]:
data = xr.concat([data0,data1,data2,data3,data4,data5], dim='time').sortby('time')

## Open predictions

In [ ]:
nee = xr.open_dataarray('/g/data/os22/chad_tmp/NEE_modelling/results/predictions/nee_AUS_2015_2020.nc',
                       chunks=dict(x=1000,y=1000))

# Compute correlations

TODO: compute p-values and plot hashes over areas with significance > 0.05
https://xskillscore.readthedocs.io/en/stable/api/xskillscore.pearson_r_eff_p_value.html

In [ ]:
data = data.sel(time=nee.time) #match times

In [ ]:
precip_corr = xr.corr(nee, data['rain'], dim='time').compute()
lai_corr = xr.corr(nee, data['lai'], dim='time').compute()
sm_corr = xr.corr(nee, data['soil_moisture'], dim='time').compute()
tavg_corr = xr.corr(nee, data['tavg'], dim='time').compute()

In [ ]:
sm_corr.plot.imshow(size=6, vmin=-1, vmax=1, cmap='RdBu')
plt.title('Pearson correlation: NEE & Soil Moisture');

In [ ]:
lai_corr.plot.imshow(size=6, vmin=-1, vmax=1, cmap='RdBu')
plt.title('Pearson correlation: NEE & Leaf Area Index');

In [ ]:
tavg_corr.plot.imshow(size=6, vmin=-1, vmax=1, cmap='RdBu')
plt.title('Pearson correlation: NEE & Average Temperature');

In [ ]:
precip_corr.plot.imshow(size=6, vmin=-1, vmax=1, cmap='RdBu')
plt.title('Pearson correlation: NEE & Precipitation');

## Plot relationships between NEE, GPP, ER and environmental variables (P, T, SM etc)

Following Lui et al. (2018) 